### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [4]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(type(doc))
    pprint(vars(doc))
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '아메리카노',
              'menu_number': 1,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '1. 아메리카노\n'
                 '   • 가격: ₩4,500\n'
                 '   • 주요 원료: 에스프레소, 뜨거운 물\n'
                 '   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 '
                 '잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.',
 'type': 'Document'}

메뉴 번호: 1
메뉴 이름: 아메리카노
내용:
1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 ...
<class 'langchain_core.documents.base.Document'>
{'id': None,
 'metadata': {'menu_name': '카페라떼',
              'menu_number': 2,
              'source': '../data/cafe_menu_data.txt'},
 'page_content': '2. 카페라떼\n'
                 '   • 가격: ₩5,500\n'
                 '   • 주요 원료: 에스프레소, 스팀 밀크\n'
                 '   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 '
    

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3:latest") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("../db/cafe_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

In [6]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [7]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema
)

C:\Users\user\AppData\Local\Temp\ipykernel_2784\2894064045.py:24: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_search = runnable.as_tool(


In [8]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [9]:
# 위키 검색 실행
query = "커피 역사"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='커피의 역사는 동아프리카로서부터 시작해, 중동, 유럽, 인도등으로 퍼지면서 오늘날 전 세계적으로 커피가 퍼진 기록을 가리킨다.


== 어원 ==
커피(coffee)라는 말은 아랍어 카흐와(قهوة)에서 오스만 터키어 카흐베(kahve)로, 여기서 다시 네덜란드어로 코피(koffie)로 간 것이 1582년에 영어로 들어간 것이다.
아랍어 '카흐와'는 원래 포도주의 한 종류를 가리키는 용어로, 식욕을 억제하는 까닭에 '배고픔을 덜다'는 의미의 '카하'(قها)에서 유래한 것이라고 받아들여진다. 다른 견해로는 '힘', '에너지'를 의미하는 '쿠으와'에서 왔다거나, 중세 에티오피아에 위치했던 카파 왕국이 커피를 아랍 세계에 전해주었기 때문에 거기에서 유래했다는 의견이 있다.


== 커피의 시작 ==
커피가 확실하게 어디에서 유래되었는지에 대해서는 기록이 없다. 그러나 커피나무는 동아프리카의 에티오피아의 카파 주에서 발견되었고 서기 850년쯤에 시작된 것으로 증명되었으나, 어떤 사람들은 중동의 예멘에서 서기 575년경에 시작되었다고 주장한다. 에티오피아의 산 속에 있던 유목민족들이 커피 음료 대신 커피 열매를 통째로 먹었을거라고 하며, 에티오피아의 갈라 족(오늘날 오로모 족)은 이 커피 나무 열매와 동물 비계를 섞어 먹었는데 커피를 이런 방식으로 섭취하는 문화는 에티오피아에서 시작되었던 것이 확실하다고 볼 수 있다. 에티오피아에서 처음 커피를 발견했을 때 '번' (ቡና)이라고 불렀으며, 커피 열매를 다양하기 섭취하기 위해 퀴시르(قشر)라는 달콤한 커피 와인이나 돼지 비계와 커피를 섞은 간식처럼 커피를 이용한 요리를 많이 만들었다.


=== 기원에 대한 다양한 설 ===

커피의 기원에 대해 가장 인정받고 있으며 가장 오래된 이야기는 11세기에 아비시니아 제국(현재 에티오피아)에서 유래했다는 설이다 (어떤 설은 서기 850년이라고도 주장한다.). 에티오피아의 고지대에서 염소떼를 방목하는 칼디라는 목동이 있었는데, 저녁때가 되면 돌아오는 염

In [10]:
from langchain_core.tools import tool

# menu db 벡터 저장소 로드
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_cafe_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized cafe menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = cafe_db.similarity_search(query, k=4)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_cafe_func))
print("-"*100)

print("name: ")
print(db_search_cafe_func.name)
print("-"*100)

print("description: ")
pprint(db_search_cafe_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_cafe_func.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
db_search_cafe_func
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized cafe menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized cafe menu information '
                'from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'db_search_cafe_func',
 'type': 'object'}
------------

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [12]:
# LLM에 도구를 바인딩 (3개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, search_wiki, db_search_cafe_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "아메리카노의 가격과 특징은 무엇인가요?"
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AOyGzsQbfwvhzqM8M0rNGGYf', 'function': {'arguments': '{"query":"아메리카노"}', 'name': 'db_search_cafe_func'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 139, 'total_tokens': 160, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bj2anJndjdT1h8iiKTtyP61rtZXF8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fd6df10d-9122-4374-a3d6-33b7f3fec317-0', tool_calls=[{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_AOyGzsQbfwvhzqM8M0rNGGYf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 139, 'output_tokens': 21, 'total_tokens': 160, 'input_to

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)         

    print("tool_msgs: \n")
    for tool in tool_msgs:
        pprint(tool.name)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


In [14]:
# 체인 실행
response = cafe_menu_chain.invoke("아메리카노의 가격은 무엇인가요?")

# 응답 출력 
print(response.content)

('db_search_cafe_func: \n'
 "{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': "
 "'call_tJ4rMeGxoXE2qaPra8oNvXBL', 'type': 'tool_call'}")
----------------------------------------------------------------------------------------------------
tool_msgs: 

'db_search_cafe_func'
----------------------------------------------------------------------------------------------------
아메리카노의 가격은 ₩4,500입니다. 

또한, 아이스 아메리카노의 가격도 동일하게 ₩4,500입니다.


In [15]:
# 체인 실행
response = cafe_menu_chain.invoke("아메리카노의 가격과 특징은 무엇인가요?")

# 응답 출력 
print(response.content)

('db_search_cafe_func: \n'
 "{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': "
 "'call_exIT4WcsEb8pglMjhlU7O5Bc', 'type': 'tool_call'}")
----------------------------------------------------------------------------------------------------
tool_msgs: 

'db_search_cafe_func'
----------------------------------------------------------------------------------------------------
아메리카노에 대한 정보는 다음과 같습니다:

### 아메리카노
- **가격:** ₩4,500
- **주요 원료:** 에스프레소, 뜨거운 물
- **설명:** 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 원하실 경우 설탕이나 시럽을 추가할 수 있습니다.

### 아이스 아메리카노
- **가격:** ₩4,500
- **주요 원료:** 에스프레소, 차가운 물, 얼음
- **설명:** 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 커피입니다. 깔끔하고 시원한 맛이 특징이며, 여름철에 인기가 높습니다.

아메리카노는 커피의 진한 맛을 즐기고 싶은 분들에게 적합한 음료입니다.


### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"커피 역사"})
pprint(summarized_text)

('커피의 역사는 동아프리카에서 시작되어 중동, 유럽, 인도 등으로 퍼져나갔으며, 현재 전 세계적으로 소비되고 있다. "커피"라는 단어는 '
 '아랍어 "카흐와"에서 유래하였고, 커피의 기원에 대한 여러 설이 존재한다. 에티오피아의 칼디라는 목동이 염소들이 커피 열매를 먹고 활력을 '
 '얻는 것을 보고 커피를 발견했다는 전설이 유명하다. 15세기 중반, 예멘에서 현대적인 커피 음료가 처음으로 만들어졌고, 이후 중동과 '
 '유럽으로 확산되었다. 커피는 다양한 방법으로 준비되며, 세계에서 가장 많이 소비되는 음료 중 하나로, 주로 아메리카, 동남아시아, '
 '아프리카에서 재배된다. 커피 산업은 경제적으로 중요한 반면, 생산자들은 종종 빈곤에 시달리고 있으며, 환경 문제도 제기되고 있다.')


In [19]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [24]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요.", name="example_user"),
    AIMessage("메뉴 검색과 위키피디아 검색을 진행하겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}]),
    ToolMessage("아메리카노: 가격 ₩4,500, 에스프레소에 뜨거운 물을 더함", tool_call_id="1"),    
    AIMessage("아메리카노의 가격은 ₩4,500이며, 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}]),
    ToolMessage("커피가 확실하게 어디에서 유래되었는지에 대해서는 기록이 없습니다. 그러나 커피나무는 동아프리카의 에티오피아의 카파 주에서 발견되었고 \
                서기 850년쯤에 시작된 것으로 증명되었으나, 어떤 사람들은 중동의 예멘에서 서기 575년경에 시작되었다고 주장합니다.", tool_call_id="2"),
    AIMessage("아메리카노(₩4,500)는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다.\
               커피의 역사에 관해 커피가 확실하게 어디에서 유래되었는지에 대해서는 기록이 없습니다. 그러나 커피나무는 동아프리카의 에티오피아의 카파 주에서 발견되었고 \
                서기 850년쯤에 시작된 것으로 증명되었으나, 어떤 사람들은 중동의 예멘에서 서기 575년경에 시작되었다고 주장합니다.", name="example_assistant"),
]

system = """You are an AI assistant providing cafe menu information and general coffee-related knowledge.
For information about the cafe's menu, use the db_search_cafe_func tool.
For other general information, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "아메리카노 정보와 커피 역사를 알려주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'db_search_cafe_func', 'args': {'query': '아메리카노'}, 'id': 'call_of0Wd4WinBYVNtDiq5W4qOzx', 'type': 'tool_call'}
{'name': 'wiki_summary', 'args': {'query': '커피 역사'}, 'id': 'call_IJwSRJ0RIbXgFMaQW3xkHYo8', 'type': 'tool_call'}


In [26]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing cafe menu information and general coffee-related knowledge.
For information about the cafe's menu, use the db_search_cafe_func tool.
For other general information, use the wiki_summary tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools


In [48]:
# 도구 실행 체인 정의
@chain
def cafe_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        pprint(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_cafe_func":
            tool_message = db_search_cafe_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)         

    print("tool_msgs: \n")
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)



# 체인 실행
query = "아메리카노 정보와 커피 역사를 알려주세요."
response = cafe_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)


tool_msgs: 

----------------------------------------------------------------------------------------------------
('아메리카노는 가격이 ₩4,500이며, 진한 에스프레소에 뜨거운 물을 더해서 만들어진 블랙 커피입니다.\n'
 '\n'
 '커피의 역사는 정확한 기원이 밝혀지지 않았지만, 일반적으로 에티오피아의 카파 주에서 시작된 것으로 여겨집니다. 초기에는 서기 850년경에 '
 '커피나무가 발견되었고, 일부 역사가들은 중동의 예멘에서 서기 575년경에 커피가 사용되기 시작했다고 주장하기도 합니다. 그렇게 커피는 전 '
 '세계로 퍼져 다양한 문화 속에서 사랑받는 음료가 되었습니다.')
